In [1]:
import pandas as pd
import sklearn as sfs
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('..')
from model_handler import ModelHandler
from feature_selection import FeatureSelectionAndGeneration
handler = ModelHandler()
dataset = handler.dataset
train_set = dataset[handler.train_mask]

The dataset includes different risks that need a prediction. Every risk is considered as a different target of labels, namely a response variable.

The aim is to build a model able to predict each risk in the most accurate way possible. However, the learning process is different for each of them, meaning that the minimum set of variables that best explain the largest amount of variance in the dataset is unique for every risk. As a consequence, the following pipeline will be executed as much time as the number of risks in order to return as more precise predictions as possible. 

# Dataset splitting

The first step consists in splitting the dataset into training and test sets. The first will be used during the feature selection part, which is implemented using a boosted logistic regression model. This is a supervised learning approach, thus labels are needed for the regression to be carried out. In this dataset risks are assigned to only some of the cities, therefore it's wise to select as training set all the entries containing values for the given risk. All the rest will be referred to as test set, used for the classification task, since those cities will be the ones needing a prediction.

# Feature selection

When there is a highly non-linear and complex relationship between the predictors and the labels decision trees are preferable. The dataset has many different predictors and we don't know whether this relationship is linear or not.

The most robust approach among the ensemble method is `Boosting`. It allows to aggregate many decision trees, differently from `Random Forest`, and grow them sequentially, instead of using boostrap sampling like in `Bagging`. 

The procedure consists in fitting small trees to the residuals in order to slowly improve the prediction error. Generally, model that learn slowly tend to perform better. A pitfall of Boosting, however, is that it relies very much on its tuning parameters. Hence, it's important to undergo `Cross Validation` in order to select the combination returning the highest accuracy, for every target. 
For this purpose we decided to use 10-fold cross validation in such a way to speed up the tuning process, which is already slow given the amount of parameters that need to be optimized.

In [3]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.pipeline import Pipeline
import shutil
import os
memory_dir = '.pipeline_cache.tmp'

XgBoost has as default objective function `reg:squarederror`, which corresponds to a linear regression with mean-squared error as loss function.

In [4]:
!pip install bayesian-optimization

You should consider upgrading via the '/home/vaslem/code/.venv/bin/python3.9 -m pip install --upgrade pip' command.


In [51]:
from bayes_opt import BayesianOptimization
if os.path.isdir(memory_dir):
    shutil.rmtree(memory_dir)

def init_model(**model_params):
    return Pipeline([('generation_and_selection', FeatureSelectionAndGeneration(feats_num=200)), ('regressor', xgb.XGBRegressor(**model_params))],memory=memory_dir)
    



# def boosting_reg(train, y_train, risk, best_parameters):
    
#     '''Cross Validation'''

In [57]:
from sklearn.model_selection import cross_val_score
from data.labeled.preprocessed import RISKS_MAPPING
optimal_params = {}
CONSTANTS = {'subsample': 0.8, 'objective':"reg:squarederror"}
for (risk, total_set, [train_set, valid_set]) in handler.get_total_train_val_set_per_risk():
    print(f"\n\n**Risk: {RISKS_MAPPING[risk]}**")
    print(f"Annotated Samples Size: {total_set.shape[0]}\n")
    def xgb_evaluate(max_depth, 
                     gamma, 
                     alpha,
                     colsample_bytree, n_estimators):
        params = {'max_depth': int(max_depth),
                  'subsample': 0.8,
                  'alpha': alpha,
                  'gamma': gamma,
                  'colsample_bytree': colsample_bytree,
                   'n_estimators': int(n_estimators)}
        params.update(CONSTANTS)

        model = init_model(**params)
        train_tuple = (total_set[handler.feat_names], total_set[risk])
        reg_cv = model.fit(*train_tuple)
        cv_result = np.mean(cross_val_score(model, *train_tuple, cv=3,scoring='neg_mean_squared_error'))
        return cv_result
    xgb_bo = BayesianOptimization(xgb_evaluate, {'max_depth': (1, 7), 
                                                 'alpha': (0,20),
                                                 'gamma': (0, 1),
                                                 'colsample_bytree': (0.3, 0.9),
                                                 "n_estimators":[200,1000],
                                                }
                                  
                                 )
    
    # Use the expected improvement acquisition function to handle negative numbers
    # Optimally needs quite a few more initiation points and number of iterations
    xgb_bo.maximize(init_points=10, n_iter=10)
    params = xgb_bo.max['params']
    params['max_depth'] = int(params['max_depth'])
    params['n_estimators'] = int(params['n_estimators'])
    params.update(CONSTANTS)
    optimal_params[risk] = params



**Risk: Higher water prices**
Samples Size: 87
|   iter    |  target   |   alpha   | colsam... |   gamma   | max_depth | n_esti... |
-------------------------------------------------------------------------------------
|  1        | -1.029    |  10.69    |  0.6609   |  0.8468   |  6.822    |  474.6    |
|  2        | -1.057    |  8.866    |  0.4458   |  0.9086   |  1.264    |  577.8    |
|  3        | -1.02     |  12.46    |  0.3204   |  0.4169   |  3.166    |  934.4    |
|  4        | -1.062    |  11.14    |  0.7036   |  0.2152   |  2.133    |  483.0    |
|  5        | -1.099    |  3.302    |  0.7593   |  0.5758   |  4.68     |  407.2    |
|  6        | -1.261    |  4.197    |  0.3899   |  0.6167   |  3.708    |  355.4    |
|  7        | -1.543    |  0.7062   |  0.4681   |  0.9942   |  1.768    |  420.4    |
|  8        | -1.065    |  16.42    |  0.544    |  0.1535   |  5.441    |  481.7    |
|  9        | -1.306    |  4.866    |  0.5051   |  0.2079   |  5.595    |  869.3    |
|  10

|  13       | -0.3317   |  13.77    |  0.3929   |  0.2846   |  3.817    |  525.9    |
|  14       | -0.3744   |  1.682    |  0.6329   |  0.02555  |  6.661    |  517.7    |
|  15       | -0.3286   |  9.322    |  0.4649   |  0.3631   |  3.073    |  530.0    |
|  16       | -0.3459   |  2.841    |  0.5927   |  0.5091   |  1.658    |  532.3    |
|  17       | -0.3276   |  9.349    |  0.4026   |  0.2734   |  6.366    |  528.0    |
|  18       | -0.3298   |  12.92    |  0.8218   |  0.9314   |  4.781    |  809.7    |
|  19       | -0.3396   |  5.678    |  0.6093   |  0.3203   |  1.765    |  807.0    |
|  20       | -0.3583   |  19.56    |  0.8028   |  0.4251   |  6.82     |  813.8    |


**Risk: Declining water quality**
Samples Size: 183
|   iter    |  target   |   alpha   | colsam... |   gamma   | max_depth | n_esti... |
-------------------------------------------------------------------------------------
|  1        | -1.017    |  15.24    |  0.5021   |  0.4765   |  4.03     |  955.4    |


|  4        | -0.6495   |  12.97    |  0.7692   |  0.7693   |  6.901    |  950.7    |
|  5        | -0.6468   |  15.29    |  0.3974   |  0.7186   |  4.209    |  551.4    |
|  6        | -0.7587   |  5.216    |  0.8815   |  0.8894   |  4.746    |  758.1    |
|  7        | -0.6468   |  18.41    |  0.3315   |  0.4609   |  5.848    |  485.1    |
|  8        | -0.6468   |  13.9     |  0.5407   |  0.595    |  3.143    |  348.1    |
|  9        | -0.6468   |  16.1     |  0.4538   |  0.407    |  1.782    |  290.3    |
|  10       | -0.8251   |  4.691    |  0.6606   |  0.373    |  5.543    |  788.6    |
|  11       | -0.9881   |  0.0      |  0.9      |  0.9517   |  1.0      |  618.7    |
|  12       | -0.8716   |  0.3495   |  0.4567   |  0.1852   |  4.343    |  213.2    |
|  13       | -0.9698   |  2.446    |  0.8587   |  0.3136   |  3.82     |  999.7    |
|  14       | -0.6468   |  12.5     |  0.664    |  0.9763   |  2.782    |  348.4    |
|  15       | -0.6468   |  20.0     |  0.4687   |  1.0

In [66]:
from data.model import MODEL_BEST_PARAMS_PATH
pd.DataFrame(optimal_params).to_csv(MODEL_BEST_PARAMS_PATH)